In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os
from datasets import ClassLabel, Sequence

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = "cuda"
model_name_or_path = "gpt2"
tokenizer_name_or_path = "gpt2"

text_column = "sentence"
label_column = "text_label"
max_length = 256
lr = 1e-2
num_epochs = 30
batch_size = 8

In [8]:
from datasets import load_dataset



dataset = load_dataset('csv', data_files={'train': "training_data.csv",
                                              'test': 'testing_data.csv'})

# OR get them from the dataset
label_names = sorted(set(labels for labels in dataset["train"]["label"]))



# print(dataset)
dataset["validation"] = dataset["test"]
del dataset["test"]


classes = label_names



dataset = dataset.map(
    remove_columns=["url", "index"],
    batched=True,
    num_proc=1,
)

# dataset["validation"][0]
# # {"sentence": "Profit before taxes was EUR 4.0 mn , down from EUR 4.9 mn .", "label": 0, "text_label": "negative"}

##########################################################################################
# from datasets import load_dataset

# dataset = load_dataset("financial_phrasebank", "sentences_allagree")
# dataset = dataset["train"].train_test_split(test_size=0.1)
# dataset["validation"] = dataset["test"]
# del dataset["test"]

# classes = dataset["train"].features["label"].names
# dataset = dataset.map(
#     lambda x: {"text_label": [classes[label] for label in x["label"]]},
#     batched=True,
#     num_proc=1,
# )

# dataset["train"][0]
# {"sentence": "Profit before taxes was EUR 4.0 mn , down from EUR 4.9 mn .", "label": 0, "text_label": "negative"}

Found cached dataset csv (C:/Users/WC/.cache/huggingface/datasets/csv/default-ac57974707e70320/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\WC\.cache\huggingface\datasets\csv\default-ac57974707e70320\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-a1cfc518ad1f5f7e.arrow
Loading cached processed dataset at C:\Users\WC\.cache\huggingface\datasets\csv\default-ac57974707e70320\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-562c29aa53950ec9.arrow


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=20, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

C:\Users\WC\anaconda3\envs\preprocessing\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\WC\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/381 [00:00<?, ? examples/s]

Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [20]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [21]:
# peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)
peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, num_virtual_tokens=20)

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"trainable params: 983040 || all params: 738651136 || trainable%: 0.13308583065659835"

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.LlamaConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [8]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|███████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:55<00:00,  1.24s/it]


epoch=0: train_ppl=tensor(2656.1096, device='cuda:0') train_epoch_loss=tensor(7.8846, device='cuda:0') eval_ppl=tensor(13627.4336, device='cuda:0') eval_epoch_loss=tensor(9.5198, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:56<00:00,  1.25s/it]


epoch=1: train_ppl=tensor(735.7351, device='cuda:0') train_epoch_loss=tensor(6.6009, device='cuda:0') eval_ppl=tensor(2793.1575, device='cuda:0') eval_epoch_loss=tensor(7.9349, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:55<00:00,  1.24s/it]


...

epoch=27: train_ppl=tensor(3.2209, device='cuda:0') train_epoch_loss=tensor(1.1697, device='cuda:0') eval_ppl=tensor(4.9675, device='cuda:0') eval_epoch_loss=tensor(1.6029, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:01<00:00,  1.36s/it]


epoch=28: train_ppl=tensor(3.1581, device='cuda:0') train_epoch_loss=tensor(1.1500, device='cuda:0') eval_ppl=tensor(4.9403, device='cuda:0') eval_epoch_loss=tensor(1.5974, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:00<00:00,  1.35s/it]

epoch=29: train_ppl=tensor(3.0788, device='cuda:0') train_epoch_loss=tensor(1.1245, device='cuda:0') eval_ppl=tensor(4.9311, device='cuda:0') eval_epoch_loss=tensor(1.5956, device='cuda:0')


In [10]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")


accuracy=4.430379746835443 % on the evaluation dataset
eval_preds[:10]=['Remote Services:     ', 'Network Discovery Discoveryg... Un  A  ', 'Network Service Discovery A A    ', 'Remote Service Discovery   A A ', 'Command Service Discovery      ', 'Network Service Discovery Au Au Au', 'Command and Scripting Interpreter:   In', 'Command and Scripting Interpreter:          ', 'Explo and Scripting Interpreter:s', 'Remote and Scripting Interpreter:..']
dataset['validation']['text_label'][:10]=['Remote Services', 'Network Service Discovery', 'Network Service Discovery', 'Network Service Discovery', 'Network Service Discovery', 'Network Service Discovery', 'Command and Scripting Interpreter', 'Command and Scripting Interpreter', 'Command and Scripting Interpreter', 'Command and Scripting Interpreter']


In [11]:
for pred, true in zip(eval_preds, dataset["validation"]["sentence"]):
    print("pred", pred)
    print("true", true)
    break

pred Remote Services:     
true This text describes the opening of a command shell session between two devices, with one device having an IP address of 192.168.0.186 and the other having an IP address of 192.168.0.130. The session was established on August 10th, 2019 at 20:39:54 IST (Indian Standard Time). The user associated with the session is identified by their username "unm" and they belong to a group called "wim"..
